# 5 Merging Datasets

Contents

1. Importing Libraries
2. Importing Data
    2.1 Data Wrangling & Consistency checks on df_ords_prior
3. Checking and Merging Dataframes 
4. Export Merged Dataframe

#01 Importing Libraries

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os

#02 Importing Data

In [2]:
#Folder Path
path = r'C:\Users\Caroline\Documents\Career Foundry\03 2023 Instacart Basket Analysis'

In [3]:
#import orders_checked.csv as df_ords
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), index_col = 0)

In [16]:
#import order_products_prior.csv as df_ords_prior
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'order_products__prior.csv'), index_col = False)

#02.1 Data Wrangling & Consistency checks on df_ords_prior

In [26]:
#  Missing values check
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [27]:
# Checking for full duplicates
df_dups = df_ords_prior[df_ords_prior.duplicated()]

In [28]:
df_dups

,order_id,product_id,add_to_cart_order,reordered


In [29]:
# Checking for mixed data types in df_ords_prior
for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col)
print ('test complete - this statement will always print')

test complete - this statement will always print


In [30]:
#Descriptive statistics of df_ords_prior
df_ords_prior.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [31]:
# Confirming the shape of in df_ords_prior
df_ords_prior.shape

(32434489, 4)

Data looks consistent and without null values, duplicates, and mixed data types. The 'add_to_cart_order' column could possibly be dropped but will be kept for now.

#03 Checking and Merging Dataframes

In [17]:
#check first rows of df_ords_prior
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [14]:
#check first rows of df_ords
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [18]:
#Check df_ords_prior dataframe size
df_ords_prior.shape

(32434489, 4)

In [19]:
# Check df_ords dataframe size
df_ords.shape

(3421083, 6)

In [20]:
# Merge df_ords and df_ords_prior. Indicator = true to check if data is present in both dataframes
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [21]:
#Check output of merged dataframes
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [22]:
# Check value count for '_merge' column
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

In [24]:
#Check df_merged_large for file size and number of rows
df_merged_large.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   _merge                  category
dtypes: category(1), float64(1), int64(8)
memory usage: 2.4 GB


#04 Export Merged Dataframe

In [25]:
#Export df_merged_large in .pkl format
df_merged_large.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))